In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Overview
This notebook uses a modified version of Inception V3 to distinguish between photos of dogs and cats.

Training accuracy after a few epochs is about 99% while validation accuracy is around 96% so there is some overfitting going on.

This notebook must be run with GPU acceleration because of the inclusion of the large Inception network.

Image preprocessing consists of resizing to 150 x 150 and rescaling the pixel values by dividing by 255.  Resizing happens before training while pixel rescaling happens on-the-fly during training.  Performing resizing during training increases the load on the CPU during training (for every epoch), which is why it is done before training (only once).

It does not matter much as the above mentioned accuracy figures are already achieved after only a few epochs, which only take a few minutes of training time.

The CPU still bottlenecks training.  This is likely caused by the repetitive fetching of data and training of the data.

In [ ]:
%matplotlib inline

import zipfile
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import shutil

import tensorflow as tf
from tensorflow.keras import utils
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator

from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import RMSprop

Check for availability and use of GPU.

In [ ]:
!nvidia-smi
print(tf.test.is_gpu_available())
print(tf.test.gpu_device_name())
print(tf.config.experimental.list_physical_devices(device_type='GPU'))

Empty the /kaggle/working directory so you can run this notebook multiple times.

In [ ]:
working_dir = "/kaggle/working/"
for rootdir, dirs, files in os.walk(working_dir):
    for subdir in dirs:
        shutil.rmtree(subdir)

Unzip the training set in the working directory.

In [ ]:
input_dir = "/kaggle/input/dogs-vs-cats/"

local_zip = input_dir + 'train.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall(working_dir)
zip_ref.close()

Unzip the test set.

In [ ]:
local_zip = input_dir + 'test1.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall(working_dir)
zip_ref.close()

Do not try to see the files in the training or test set in the data panel on the right as it will hang the browser.

Print the names of a few files in the training set.

In [ ]:
train_dir = working_dir + 'train/'
for dirname, _, filenames in os.walk(train_dir):
    for filename in filenames[:3]:
        print(os.path.join(dirname, filename))

Print the names of a few files in the test set.

In [ ]:
test_dir = working_dir + 'test1/'
for dirname, _, filenames in os.walk(test_dir):
    for filename in filenames[0:3]:
        print(os.path.join(dirname, filename))

Plot a few training images.

In [ ]:
image_files = ["/kaggle/working/train/dog.918.jpg",
               "/kaggle/working/train/dog.7156.jpg",
               "/kaggle/working/train/dog.10335.jpg"]

for i, img_dir in enumerate(image_files):
  img = mpimg.imread(img_dir)
  plt.imshow(img)
  plt.axis('Off')
  plt.show()

Print the number of training and test images.

In [ ]:
print(len(os.listdir(train_dir)))
print(len(os.listdir(test_dir)))

# Separate Resizing
Resize the training images and put the results in a new training directory.

In [ ]:
destination_dir = working_dir + 'train_resized/'
if (os.path.isdir(destination_dir) == False):
    os.mkdir(destination_dir)

for dirname, _, filenames in os.walk(train_dir):
    for filename in filenames:
        input_path = os.path.join(dirname, filename)
        img = image.load_img(input_path, target_size=(150, 150))
        x = image.img_to_array(img)
#         x = np.divide(x, 255.0)
        output_path = os.path.join(destination_dir, filename)
        utils.save_img(output_path, x, data_format='channels_last', file_format='jpeg', scale=False)
        
train_dir = working_dir + 'train_resized/'

Move the training images to separate subfolders so we can use an ImageDataGenerator and have it automatically create the labels from the subfolder names.

In [ ]:
dog_files = [name for name in os.listdir(train_dir) if (name.split('.')[0]=='dog')]
print(dog_files[:3])
print(len(dog_files))
cat_files = [name for name in os.listdir(train_dir) if (name.split('.')[0]=='cat')]
print(cat_files[:3])
print(len(cat_files))

In [ ]:
train_subs_dir = working_dir + 'train_subs/'
if (os.path.isdir(train_subs_dir) == False):
    os.mkdir(train_subs_dir)

train_subs_dog_dir = train_subs_dir + 'dog/'
if (os.path.isdir(train_subs_dog_dir) == False):
    os.mkdir(train_subs_dog_dir)

train_subs_cat_dir = train_subs_dir + 'cat/'
if (os.path.isdir(train_subs_cat_dir) == False):
    os.mkdir(train_subs_cat_dir)

print(train_subs_dir)
print(train_subs_dog_dir)
print(train_subs_cat_dir)

for name in dog_files:
    shutil.copyfile(train_dir + name, train_subs_dog_dir + name)
    
for name in cat_files:
    shutil.copyfile(train_dir + name, train_subs_cat_dir + name)

In [ ]:
for dirname, _, filenames in os.walk(train_subs_dog_dir):
    for filename in filenames[:3]:
        print(os.path.join(dirname, filename))

for dirname, _, filenames in os.walk(train_subs_cat_dir):
    for filename in filenames[:3]:
        print(os.path.join(dirname, filename))

In [ ]:
print(len(os.listdir(train_subs_dog_dir)))
print(len(os.listdir(train_subs_cat_dir)))

Use the last 2500 dog images and the last 2500 cat images as the validation set.

In [ ]:
val_subs_dir = working_dir + 'val_subs/'
if (os.path.isdir(val_subs_dir) == False):
    os.mkdir(val_subs_dir)

val_subs_dog_dir = val_subs_dir + 'dog/'
if (os.path.isdir(val_subs_dog_dir) == False):
    os.mkdir(val_subs_dog_dir)

val_subs_cat_dir = val_subs_dir + 'cat/'
if (os.path.isdir(val_subs_cat_dir) == False):
    os.mkdir(val_subs_cat_dir)

In [ ]:
for name in dog_files[:2500]:
    os.rename(train_subs_dog_dir + name, val_subs_dog_dir + name)
for name in cat_files[:2500]:
    os.rename(train_subs_cat_dir + name, val_subs_cat_dir + name)

In [ ]:
print(train_subs_dog_dir)
print(len(os.listdir(train_subs_dog_dir)))
for dirname, _, filenames in os.walk(train_subs_dog_dir):
    for filename in filenames[:3]:
        print(os.path.join(dirname, filename))

print(train_subs_cat_dir)
print(len(os.listdir(train_subs_cat_dir)))
for dirname, _, filenames in os.walk(train_subs_cat_dir):
    for filename in filenames[:3]:
        print(os.path.join(dirname, filename))

print(val_subs_dog_dir)
print(len(os.listdir(val_subs_dog_dir)))
for dirname, _, filenames in os.walk(val_subs_dog_dir):
    for filename in filenames[:3]:
        print(os.path.join(dirname, filename))

print(val_subs_cat_dir)
print(len(os.listdir(val_subs_cat_dir)))
for dirname, _, filenames in os.walk(val_subs_cat_dir):
    for filename in filenames[:3]:
        print(os.path.join(dirname, filename))

Create the image data generators.

In [ ]:
train_subs_dir = working_dir + 'train_subs/'
val_subs_dir = working_dir + 'val_subs/'

training_datagen = ImageDataGenerator(
      rescale = 1./255,
#       rotation_range=20,
#       width_shift_range=0.2,
#       height_shift_range=0.2,
#       shear_range=0.2,
#       zoom_range=0.2,
#       horizontal_flip=True,
#       fill_mode='nearest'
)

train_generator = training_datagen.flow_from_directory(
    train_subs_dir,
    target_size=(150,150),
    class_mode='binary',
    batch_size=128
)

validation_datagen = ImageDataGenerator(rescale = 1./255)

validation_generator = validation_datagen.flow_from_directory(
    val_subs_dir,
    target_size=(150,150),
    class_mode='binary',
    batch_size=128
)

Create the model, starting with Inception V3.

In [ ]:
pre_trained_model = InceptionV3(input_shape = (150, 150, 3),
                                include_top = False, 
                                weights = "imagenet") 

for layer in pre_trained_model.layers:
    layer.trainable = False

# pre_trained_model.summary()

In [ ]:
last_desired_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape: ', last_desired_layer.output_shape)
last_output = last_desired_layer.output
print('last layer output: ', last_output)

x = layers.Flatten()(last_output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.2)(x)  
x = layers.Dense  (1, activation='sigmoid')(x)        

model = Model(inputs=pre_trained_model.input, outputs=x)

Create callbacks.

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.99):
      print("\nReached 99% accuracy, cancelling training")
      self.model.stop_training = True

Compile the model.

In [ ]:
model.compile(optimizer = RMSprop(learning_rate=0.0001), 
                loss = 'binary_crossentropy',
                metrics = ['accuracy'])

# model.compile(loss = 'categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

Train the model.

In [ ]:
callbacks = myCallback()
model.fit(train_generator,
          validation_data = validation_generator,
          epochs = 100,
          batch_size=128,
          verbose = 2,
          callbacks=callbacks)

# Results
Training accuracy exceeds the 99% threshold after finishing the 4rd epoch, which is after about 3 minutes.  At that point the validation accuracy is almost 97%.

    training_datagen = ImageDataGenerator(rescale = 1./255)

    Epoch 3/100
    157/157 - 72s - loss: 0.0245 - accuracy: 0.9915 - val_loss: 0.1301 - val_accuracy: 0.9584


In [ ]:
print(len(os.listdir(test_dir)))
test_files = os.listdir(test_dir)
print(test_files[:3])
id_strings = [name.split('.')[0] for name in os.listdir(test_dir)]
test_ids = list(map(int, id_strings))
test_ids.sort()
# print(test_ids) # it looks like test_ids - ie. the numbers in the test image file names - is simply a list from 1 thru 12500
print(min(test_ids))
print(max(test_ids))
sorted_id_strings = [(test_dir+str(test_id)+'.jpg') for test_id in test_ids]
print(sorted_id_strings[:10])

In [ ]:
for name in sorted_id_strings[:10]:
    img = image.load_img(name, target_size=(150, 150))
    x = image.img_to_array(img)
    x = np.divide(x, 255.0)
    x = np.expand_dims(x, axis=0)
    
    images = np.vstack([x])
    classes = model.predict(images)
    print(name)
    print(classes[0][0])
    if classes[0][0]>0.5:
      print("Dog")
    else:
      print("Cat")
    plt.imshow(img)
    plt.axis('Off')
    plt.show()
    print()

In [ ]:
predictions = []

for name in sorted_id_strings:
    img = image.load_img(name, target_size=(150, 150))
    x = image.img_to_array(img)
    x = np.divide(x, 255.0)
    x = np.expand_dims(x, axis=0)
    
    images = np.vstack([x])
    classes = model.predict(images)
    if classes[0][0]>0.5:
      prediction = 1 # Dog
    else:
      prediction = 0 # Cat
    predictions.append(prediction)

In [ ]:
print(len(predictions))
df = pd.DataFrame()
df['id'] = test_ids
df['label'] = predictions
print(df)

In [ ]:
df.to_csv('submission.csv', index=False)